In [ ]:
#constants that get changed when notebook is autorun
SCOUT_FOLDER_PATH = r'C:\Users\kowaleskim\Workspace\measure_the_future\mtf_data\DB2 - Childrens'

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mpcolors
import numpy as np

import load_mtf_data
import os

# Measure the Future: Heatmap Visualization
This notebook creates a heatmap of interaction data from one Measure the Future Scout.

In [ ]:
#set matplotlib to display plots in notbook output rather than new window
%matplotlib inline

In [ ]:
#use load_mtf_data python script to get data from the most recent date range subfolder
(interactions, im, subfolder_name) = load_mtf_data.from_most_recent_subfolder(SCOUT_FOLDER_PATH)

In [ ]:
print(im.format, im.size, im.mode)
plt.imshow(im, cmap="gray")

In [ ]:
interactions[0]

In [ ]:
for i in interactions: 
    i['Waypoints-MPL-Format'] = list(zip(*i['Waypoints']))
    
#zip(*[(1, 4), (2, 5), (3, 6)]) -> ([1, 2, 3], [4, 5, 6])
interactions[0]

In [ ]:
def make_colormap_alpha(points):
    """Return a LinearSegmentedColormap
    points: a list of (location, r, g, b, a) tuples
    """
    cdict = {'red': [], 'green': [], 'blue': [], 'alpha': []}
    for (location, r, g, b, a) in points:
        cdict['red'].append([location, r, r])
        cdict['green'].append([location, g, g])
        cdict['blue'].append([location, b, b])
        cdict['alpha'].append([location, a, a])
    return mpcolors.LinearSegmentedColormap('CustomMap', cdict)

cmap = make_colormap_alpha(
    [(0.0, .1, .3, .9, 0.0),
     (0.05, .1, .3, .9, 0.7),
     (0.1, .9, 0.2, .7, 1.0),
     (0.4, 0.2, .9, .7, 1.0),
     #(0.7, .6, .8, .1, 1.0),
     (1.0, .9, .7, 0.2, 1.0)])

In [ ]:
title = "\n".join([
    "Number of Interactions by Location in Image",
    "scout: " + os.path.basename(SCOUT_FOLDER_PATH),
    "dates range subfolder: " + subfolder_name
])

In [ ]:
#get axes object from pyplot
ax = plt.axes()

flatten = lambda l: [item for sublist in l for item in sublist]

(counts, xedges, yedges, image) = ax.hist2d(
    #x, y coords
    flatten([i['Waypoints-MPL-Format'][0] for i in interactions]), 
    flatten([i['Waypoints-MPL-Format'][1] for i in interactions]),
    #set range to match size of image
    range=[[0, im.width], [0, im.height]],
    #set number of bins for each axis
    bins=[100, 50],
    cmap=cmap
)

plt.colorbar(
    image, #colorbar for image returned by histogram
    ax=ax, #put it on the axes object ax
    orientation="horizontal", 
    pad=0, 
    shrink=.98 #for some reason it comes out slightly larger than the image, so fix that
)

#make sure plot limits are correct
ax.set_xlim(0, 1280)
ax.set_ylim(720, 0)

#display grayscale image overlaid on histogram
ax.imshow(im, alpha=.6, cmap="gray")

#we don't really need axes labels for this
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)

ax.set_title(title)

plt.savefig(SCOUT_FOLDER_PATH + os.sep + "heatmap_" + subfolder_name + ".jpg",
           bbox_inches='tight', dpi=300)